<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Environment-Review" data-toc-modified-id="Environment-Review-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Environment Review</a></span><ul class="toc-item"><li><span><a href="#Initial-Thoughts" data-toc-modified-id="Initial-Thoughts-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initial Thoughts</a></span></li><li><span><a href="#hawking_pooled_sequences.fastq" data-toc-modified-id="hawking_pooled_sequences.fastq-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>hawking_pooled_sequences.fastq</a></span></li><li><span><a href="#getMutations.py" data-toc-modified-id="getMutations.py-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>getMutations.py</a></span></li><li><span><a href="#parseFastq.py" data-toc-modified-id="parseFastq.py-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>parseFastq.py</a></span></li></ul></li><li><span><a href="#Workflow-outline" data-toc-modified-id="Workflow-outline-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Workflow outline</a></span><ul class="toc-item"><li><span><a href="#Demultiplex-the-pooled-fastq-by-barcode" data-toc-modified-id="Demultiplex-the-pooled-fastq-by-barcode-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Demultiplex the pooled fastq by barcode</a></span></li><li><span><a href="#Dictionary-Coding-Structure" data-toc-modified-id="Dictionary-Coding-Structure-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Dictionary Coding Structure</a></span></li><li><span><a href="#Parsing-the-Parser" data-toc-modified-id="Parsing-the-Parser-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Parsing the Parser</a></span></li><li><span><a href="#Merging-Dictionarys" data-toc-modified-id="Merging-Dictionarys-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Merging Dictionarys</a></span></li><li><span><a href="#Quick-Sanity-Check" data-toc-modified-id="Quick-Sanity-Check-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Quick Sanity Check</a></span></li><li><span><a href="#Undestanding-Pileup" data-toc-modified-id="Undestanding-Pileup-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Undestanding Pileup</a></span></li></ul></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pipeline</a></span></li></ul></div>

# Week 6 Assignment

## Environment Review

In [1]:
import sys

sys.version

'3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]'

In [2]:
%%bash

ls
echo ""
cd necessary_scripts/
ls
echo ""
echo "----------------------"
echo "Line Count of getMutations"
wc -l getMutations.py
echo ""
echo "----------------------"
echo "Line Count of parseFastq.py"
wc -l parseFastq.py

Assignment 3.pdf
dgorgon_reference.fa
getMutations.py
harrington_clinical_data.txt
hawkins_pooled_sequences.fastq
necessary_scripts
parseFastq.py
__pycache__
Week6_Lay_that_Pipe.ipynb

faker.sorted.bam
faker.sorted.bam.bai
getMutations.py
parseFastq.py

----------------------
Line Count of getMutations
29 getMutations.py

----------------------
Line Count of parseFastq.py
92 parseFastq.py


### Initial Thoughts

These files are just too big, to look cat them out above, let's bring these files into jupyter

### hawking_pooled_sequences.fastq

In [3]:
%%bash

head -10 hawkins_pooled_sequences.fastq

@seq13534-419
GCAGTAGCGGTCATAAGTGGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGTGTCGTGAGGTGACGTCCGTCACTGGACGAA
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFFFFDFFDFFDDFDFDFFFFDDFFDDFDDFF
@seq86249-867
GGATTAGCGGTCATAAGTCGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGAGGTCAGTATAACCTCTCAAAGCTTTATCTACGGATGGATCCGCGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In [4]:
%%bash

head -10 harrington_clinical_data.txt

Name	Color	Barcode
Abbey	Black	GCAGA
Adelaide	Orange	CTTGA
Alana	Black	AGCTA
Arabella	Black	TTGAG
Arianna	Orange	TCCGC
Bobby	Black	CCCAC
Brayden	Black	GCAGT
Bridget	Orange	CGGTT
Bruce	Orange	ACGGA


### getMutations.py

pysam needs python 3.6 downgrading from 3.8 now

In [5]:
%%writefile getMutations.py
import pysam

#This is a slightly modified version from here: https://pysam.readthedocs.io/en/latest/api.html
# What is a pileup? It is "piling up" all the reads that have mapped to a given position of your reference.
# It is a useful way to see what reads have a mutation and what don't. 

def pileup():
    #test file, replaced with the sorted.bam you are using. Make sure it is indexed! 
    #(Use samtools index yourbam.sorted.bam)
    samfile = pysam.AlignmentFile("/home/rbif/week6/necessary_scripts/faker.sorted.bam", "rb")

    #Since our reference only has a single sequence, we're going to pile up ALL of the reads. 
    #Usually you would do it in a specific region (such as chromosome 1, position 1023 to 1050 for example)
    for pileupcolumn in samfile.pileup():
        print ("coverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        #use a dictionary to count up the bases at each position
        ntdict = {}
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                # You can uncomment the below line to see what is happening in the pileup. 
                # print ('\tbase in read %s = %s' % (pileupread.alignment.query_name, pileupread.alignment.query_sequence[pileupread.query_position]))
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                
                ########## ADD ADDITIONAL CODE HERE ############# 
                # Populate the ntdict with the counts of each base 
                # This dictionary will hold all of the base read counts per nucletoide per position.
                # Use the dictionary to calculate the frequency of each site, and report it if if the frequency is NOT  100% / 0%. 
                #############################################
        print (ntdict)
    samfile.close()

if __name__=="__main__":
    pileup()

Overwriting getMutations.py


### parseFastq.py

In [6]:
%%writefile parseFastq.py

import argparse
#Example use is 
# python parseFastq.py --fastq /home/rbif/week6/hawkins_pooled_sequences.fastq


################################################
# You can use this code and put it in your own script
class ParseFastQ(object):
    """Returns a read-by-read fastQ parser analogous to file.readline()"""
    def __init__(self,filePath,headerSymbols=['@','+']):
        """Returns a read-by-read fastQ parser analogous to file.readline().
        Exmpl: parser.next()
        -OR-
        Its an iterator so you can do:
        for rec in parser:
            ... do something with rec ...
 
        rec is tuple: (seqHeader,seqStr,qualHeader,qualStr)
        """
        if filePath.endswith('.gz'):
            self._file = gzip.open(filePath)
        else:
            self._file = open(filePath, 'rU')
        self._currentLineNumber = 0
        self._hdSyms = headerSymbols
         
    def __iter__(self):
        return self
    
    # I think this is supposed to be __next__(), so I'm changing it
    def __next__(self):
        """Reads in next element, parses, and does minimal verification.
        Returns: tuple: (seqHeader,seqStr,qualHeader,qualStr)"""
        # ++++ Get Next Four Lines ++++
        elemList = []
        for i in range(4):
            line = self._file.readline()
            self._currentLineNumber += 1 ## increment file position
            if line:
                elemList.append(line.strip('\n'))
            else: 
                elemList.append(None)
         
        # ++++ Check Lines For Expected Form ++++
        trues = [bool(x) for x in elemList].count(True)
        nones = elemList.count(None)
        # -- Check for acceptable end of file --
        if nones == 4:
            raise StopIteration
        # -- Make sure we got 4 full lines of data --
        assert trues == 4,\
               "** ERROR: It looks like I encountered a premature EOF or empty line.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber)
        # -- Make sure we are in the correct "register" --
        assert elemList[0].startswith(self._hdSyms[0]),\
               "** ERROR: The 1st line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[0],self._currentLineNumber) 
        assert elemList[2].startswith(self._hdSyms[1]),\
               "** ERROR: The 3rd line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[1],self._currentLineNumber) 
        # -- Make sure the seq line and qual line have equal lengths --
        assert len(elemList[1]) == len(elemList[3]), "** ERROR: The length of Sequence data and Quality data of the last record aren't equal.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber) 
         
        # ++++ Return fatsQ data as tuple ++++
        return tuple(elemList)
##########################################################################



if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-f", "--fastq", required=True, help="Place fastq inside here")
    args = parser.parse_args()

    #This is an example of how to use the function in your own code
    
    fastqfile = ParseFastQ(args.fastq)

    #A fastq read contains 4 lines
    for fastq_obj in fastqfile:
        #This fastq_obj is a tuple that has length of 4 and corresponds to those 4 lines
        #This is the header
        print(fastq_obj[0])
        #This is the sequence
        print(fastq_obj[1])
        #This is the separator
        print(fastq_obj[2])
        #This is the quality score
        print(fastq_obj[3])
        
        #Just an indicator showing the fastq "blocks"
        print('*'*10 + '==='*10 + '*' * 10)

Overwriting parseFastq.py


In [7]:
%%bash 

python parseFastq.py --fastq /home/rbif/week6/hawkins_pooled_sequences.fastq

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

parseFastq.py:24: DeprecationWarning: 'U' mode is deprecated
  self._file = open(filePath, 'rU')


hmmm....this must be a really big output, let me see if I can implement this in a code

## Workflow outline

Important note: We will not be able to use anything from BioPython for this assignment

### Demultiplex the pooled fastq by barcode

Create a single python script that:
1. Reads in the pooled fastq*
2. Uses the barcode from clinical data to find matches
3. Write to file
4. Trims barcode from start
5. Trims consequtive D or F from quality scores.
6. Set in foder with {name}_trimmed.fastq

### Dictionary Coding Structure

I will be using dictionarys to carry a lot of the heavy lifting inside this code. For example, I will be setting a coding_dict to make sure that everything is set in the right direction. Thus this will be more of a vinilla python programming exercise. 

In [7]:
with open("dgorgon_reference.fa") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
dgorgon_reference = [x.strip() for x in content][1]

print(dgorgon_reference)

AGCGGTCATAAGTGGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAG


### Parsing the Parser

In [9]:
from parseFastq import ParseFastQ

fastqfile = ParseFastQ("/home/rbif/week6/hawkins_pooled_sequences.fastq")
for item in fastqfile:
    #It seems like item[2] are all "+", let's just check that really quick 
    if item[2] != "+": print(item[2])

Item's 0, 1, and 3 are important to the work, so we will keep them in a dictionary.

In [10]:
fastqfile = ParseFastQ("/home/rbif/week6/hawkins_pooled_sequences.fastq")
fast_dict = {}

for item in fastqfile:
    fast_dict[item[0]] = {"tag":item[1][0:5], 
                          "seq":item[1][5:], 
                          "quality":item[3], 
                          "name":item[0]}

fast_dict['@seq13534-419']['tag']

'GCAGT'

### Merging Dictionarys 

Now we need to get these dictionarys into one final format. It would probably be best if I put these by name, but names can repeat, but I assume they would not repeat the tag. 

In [90]:
from parseFastq import ParseFastQ

#First we are going to make a dictionary with all of the clinical entries
with open("harrington_clinical_data.txt") as clinical:
    rows = (line.split('\t') for line in clinical)
    coding_dict = {row[1:][1].rstrip():{"name":row[0], "color":row[1]}  for row in rows}

# Remove the first entry, we are not going to use a Pandas style index here
coding_dict.pop('Barcode')

fastqfile = ParseFastQ("hawkins_pooled_sequences.fastq")
for i in fastqfile:
    # I don't think we need item 2 which is always a "+", so we will omit it
    coding_dict[i[1][0:5]][i[0]] = {"pretrimmed":i[1][5:], "prequality":i[3]}
    pretrim = coding_dict[i[1][0:5]][i[0]]["prequality"]

    #Search for consecutive D or F 
    trim_value = re.search(r'[DF][DF]', pretrim).start()
    coding_dict[i[1][0:5]][i[0]]["trim_value"] = trim_value
    coding_dict[i[1][0:5]][i[0]]["trimmed"] = coding_dict[i[1][0:5]][i[0]]["pretrimmed"][:trim_value]
    coding_dict[i[1][0:5]][i[0]]["quality"] = coding_dict[i[1][0:5]][i[0]]["prequality"][:trim_value]

### Quick Sanity Check

In [95]:
for i in coding_dict:
    print(coding_dict[i]["name"])

Abbey
Adelaide
Alana
Arabella
Arianna
Bobby
Brayden
Bridget
Bruce
Byron
Cali
Camilla
Case
Cason
Damien
Dania
Dereon
Destiney
Dominik
Gabrielle
Giancarlo
Greta
Israel
Javion
Jaylen
Jazlyn
Jordyn
Jordyna
Joshua
Justus
Kamden
Karlee
Kayla
Keaton
Kian
Kirsten
Libby
Lorelei
Mariela
Martin
Mckayla
Mckenzie
Natalia
Santiago
Sarai
Serena
Shelby
Sienna
Terry
Tristian


In [65]:
re.search(r'[DF][DF]',coding_dict["GCAGA"]["@seq72638-854"]["quality"]).start()

222

In [67]:
print(coding_dict["GCAGA"]["@seq72638-854"]["quality"][:222])
print(coding_dict["GCAGA"]["@seq72638-854"]["quality"][222:])

IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
DFFFFFDFDFDDFFDFDDFDFFFDFDFFDDFDDDDFFFF


According to https://support.illumina.com/bulletins/2016/04/fastq-files-explained.html 

For each cluster that passes filter, a single sequence is written to the corresponding sample’s R1 FASTQ file, and, for a paired-end run, a single sequence is also written to the sample’s R2 FASTQ file. Each entry in a FASTQ files consists of 4 lines:

1) A sequence identifier with information about the sequencing run and the cluster. The exact contents of this line vary by based on the BCL to FASTQ conversion software used. <br>
2) The sequence (the base calls; A, C, T, G and N). <br>
3) A separator, which is simply a plus (+) sign. <br>
4) The base call quality scores. These are Phred +33 encoded, using ASCII characters to represent the numerical quality scores.<br>

### Undestanding Pileup

In [2]:
import pysam

path = "fastqs/Byron.sorted.bam"

samfile = pysam.AlignmentFile(path, "rb")
i = 0

count_dict = {}
for item in samfile.pileup():
    # Mapping qualities is not important for the problem
    # print(item.get_mapping_qualities())
    
    # This is sort of useful returns alighed items
    total_count = item.get_num_aligned()
    
    # This has the names of the aligned sequences, not useful here
    # print(item.get_query_names())
    
    # Pileup column postions, not important here
    # print(item.get_query_positions())
    
    # No use for quality score pileup
    # print(item.get_query_qualities())
    
    # Jackpot
    summary = item.get_query_sequences()
    count_dict[item.pos] = {x:100*summary.count(x)/total_count for x in summary}

mutation_dict = {}
for i in count_dict.keys():
    if len(count_dict[i].keys()) > 1:
        mutation_dict[i] = count_dict[i]
        mutation_dict[i]["segments"] = total_count
#     print(item.n)
print(item.nsegments)
#     print(item.pileups)
#     print(item.pos)
#     print(item.reference_id)
#     print(item.reference_name)
#     print(item.reference_pos)
#     print(item.set_min_base_quality)
#     print(item.tid)
print(mutation_dict)   
samfile.close()

431
{83: {'G': 50.11600928074246, 'C': 49.88399071925754, 'segments': 431}}


## Pipeline

I will try to make a function for every section of the homework to help read the code

In [21]:
#%%writefile pipeline.py
from parseFastq import ParseFastQ
import re
import os
import shutil
import pysam
import datetime

class Main:
    def __init__(self):
        
        # Load reference
        with open("dgorgon_reference.fa") as f:
            dgogron = f.readlines()
        dgorgon_reference = [x.strip() for x in dgogron][1]
        
        # Create Report
        report = open("report.txt", "w")
        now = datetime.datetime.now()
        report.write("\nAnalysis Run Started:\n" + now.strftime("%c") + "\n--names--\n")
        report.close()
        
        coding_dict = self.trimmer("harrington_clinical_data.txt", "hawkins_pooled_sequences.fastq")
        self.alignment()
        for i in coding_dict:
            file_path = "fastqs/{}.sorted.bam".format(coding_dict[i]["name"])
            coding_dict[i]["mutation"] = self.pileup(file_path)
            coding_mutation = coding_dict[i]["mutation"]
            expected = dgorgon_reference[coding_mutation["position"]]
            
            # This syntax is terrible, but I am running out of time
            # for big big sets, it actually might be faster, however
            mutation = coding_mutation["possible"] - {expected}
            mutation = list(mutation)[0]
            
            with open("report.txt", "a") as myfile:
                report_text = "Sample {0} had a {1} mold, " \
                              "{2} reads, and had {3}% of the reads at " \
                              "position {4} had " \
                              "the mutation {5}. \n".format(coding_dict[i]["name"],
                                                            coding_dict[i]["color"],
                                                            coding_mutation["reads"],
                                                            round(coding_mutation[mutation],2),
                                                            coding_mutation["position"],
                                                            mutation)
                
                
                myfile.write(report_text)
        print(coding_dict[i])
        
    # Part I: Trim and report
    def trimmer(self, clinical_path, fast_path):
        #First we are going to make a dictionary with all of the clinical entries
        with open(clinical_path) as clinical:
            rows = (line.split('\t') for line in clinical)
            coding_dict = {row[1:][1].rstrip():{"name":row[0], "color":row[1]}  for row in rows}

        # Remove the first entry, we are not going to use a Pandas style index here
        coding_dict.pop('Barcode')

        fastqfile = ParseFastQ(fast_path)
        for i in fastqfile:
            # I don't think we need item 2 which is always a "+", so we will omit it

            coding_dict[i[1][0:5]][i[0]] = {"pretrimmed":i[1][5:], "prequality":i[3]}
            pretrim = coding_dict[i[1][0:5]][i[0]]["prequality"]

            #Search for consecutive D or F 
            trim_value = re.search(r'[DF][DF]', pretrim).start()
            coding_dict[i[1][0:5]][i[0]]["trim_value"] = trim_value
            coding_dict[i[1][0:5]][i[0]]["trimmed"] = coding_dict[i[1][0:5]][i[0]]["pretrimmed"][:trim_value]
            coding_dict[i[1][0:5]][i[0]]["quality"] = coding_dict[i[1][0:5]][i[0]]["prequality"][:trim_value]

        # Make a new directory unless it exists
        try:
            os.makedirs("fastqs")
            print("Made New Directory")
        except:
            print("Removing Old Directory")
            shutil.rmtree("fastqs")
            os.makedirs("fastqs")

        for listing in coding_dict:
            # First name file
            f= open("fastqs/{}_trimmed.fastq".format(coding_dict[listing]["name"]),"w+")
            for seq in coding_dict[listing]:
                # I kept the name and color in the dictionary, we can't add that to fastq file
                if seq not in {"name", "color"}:
                    f.write(seq + "\n")
                    f.write(coding_dict[listing][seq]["trimmed"] + "\n")
                    f.write("+" + "\n")
                    f.write(coding_dict[listing][seq]["quality"] + "\n")
            
        # Part 1 completed
        return coding_dict
    
    # Part 2&3 alignment
    def alignment(self):
        os.system("bwa index dgorgon_reference.fa")
        for name in os.listdir("fastqs"):
            split = name.split("_")[0]
            os.system("bwa mem dgorgon_reference.fa fastqs/{0}_trimmed.fastq > fastqs/{0}.sam".format(split))
            os.system("samtools view -bS fastqs/{0}.sam > fastqs/{0}.bam".format(split))
            os.system("samtools sort -m 100M -o fastqs/{0}.sorted.bam fastqs/{0}.bam".format(split))
            os.system("samtools index fastqs/{0}.sorted.bam".format(split))
            # Part 2&3 Completed
    
    # Part 4 
    def pileup(self,path):
        samfile = pysam.AlignmentFile(path, "rb")
        count_dict = {}
        for item in samfile.pileup():
            total_count = item.get_num_aligned()
            summary = item.get_query_sequences()
            count_dict[item.pos] = {x:100*summary.count(x)/total_count for x in summary}

        mutation_dict = {}
        for i in count_dict.keys():
            if len(count_dict[i].keys()) > 1:
                mutation_dict = count_dict[i]
                mutation_dict["possible"] = set(count_dict[i].keys())
                mutation_dict["reads"] = total_count
                mutation_dict["position"] = i
                
        samfile.close()
        return mutation_dict

if __name__== "__main__":    
    Main()

Removing Old Directory
{'name': 'Tristian', 'color': 'Black', '@seq61428-359': {'pretrimmed': 'AGCGGTCATAAGTCGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGCTCACTCAGCCATAACTATCTGTCTACTCCGTCG', 'prequality': 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFDFDDFDFDFFDFDDFFFDDDFDFDDFDDDDFFD', 'trim_value': 222, 'trimmed': 'AGCGGTCATAAGTCGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGCTCAC', 'quality': 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFIIIIII

In [18]:
{1,2} - {1}

[{2}]